In [6]:
print("# Carregando...\n")
from IPython.display import display, clear_output
import utils
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import ipywidgets as widgets
import io

uploader = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

print("Importe um arquivo .csv")
display(uploader)

# Carregando...

Importe um arquivo .csv


FileUpload(value={}, accept='.csv', description='Upload')

In [7]:
isEmpty = not bool(uploader.value)
try:
    input_file = list(uploader.value.values())[0]
    content = input_file['content']
    content = io.StringIO(content.decode('utf-8'))
    df = pd.read_csv(content, sep=';')    
    print("CSV importado com sucesso")
except:
    print("Precisa subir um arquivo .csv")

CSV importado com sucesso


In [8]:
# gerando DFs com respectivos filtros
print("# Inciando análise...")
try:
    consequente = df[(df.internacao=='sup_media')]
    consequente = consequente.reset_index(drop=True)
    inter_filter = consequente[(consequente.MP25 =='sup_media') |
                               (consequente.MP10 =='sup_media') |
                               (consequente.CO   =='sup_media') |
                               (consequente.SO2  =='sup_media') |
                               (consequente.NO2  =='sup_media') |
                               (consequente.O3   =='sup_media')]
    inter_filter = inter_filter.reset_index(drop=True)

    df_supports = utils.get_supports(utils.joinDate(inter_filter))
    df_timeAnalysis = utils.joinDate(df)

    tx_tables = utils.timeRelation(df_timeAnalysis, df_supports)

    options = []
    for row in df_supports.iloc[:, :-2].iterrows():
        index, ref_pattern = row    
        line = ref_pattern.tolist()
        line = "   --   ".join(line)
        options.append((line, index))

    pattern_widget = widgets.Box(
        [
            widgets.Label(value='Padrões encontrados:'),
            widgets.Dropdown(
                options=options,
                layout={'width': 'max-content'}
            )
        ]
    )

    gap_widget = widgets.IntSlider(
        value=0,
        min=-7,
        max=7,
        step=1,
        description='GAP:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    print("# Análise finalizada com sucesso.")
    print("\n-----------------------------------------------")
    print("Defina o gap:")
    display(gap_widget)
    print("\nEscolha o padrão para análise:")
    display(pattern_widget)


    button = widgets.Button(description="Atualizar")
    output = widgets.Output()

    display(button, output)
    state = False
    def on_button_clicked(b):    
        with output:
            clear_output(wait=True)
            pattern = pattern_widget.children[1].value
            gap = gap_widget.value
            ref = df_supports.iloc[pattern, :-2]
            tx_table = tx_tables[str(ref)][gap]

            print("\n\nTabela tx gap=%d" % (gap))
            display(tx_table)
    button.on_click(on_button_clicked)
except:
    print("Algo deu errado. \nPor favor, verifique a compatibilidade do arquivo .csv")

# Inciando análise...
# Análise finalizada com sucesso.

-----------------------------------------------
Defina o gap:


IntSlider(value=0, description='GAP:', max=7, min=-7)


Escolha o padrão para análise:


Box(children=(Label(value='Padrões encontrados:'), Dropdown(layout=Layout(width='max-content'), options=(('inf…

Button(description='Atualizar', style=ButtonStyle())

Output()